### Diffie-Hellman Key Exchange

In [1]:
import random
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes


# DIFFIE-HELLMAN KEY EXCHANGE
def diffie_hellman_key_exchange(p, g):
    # Private keys
    a = random.randint(1000, 9999)  # Alice
    b = random.randint(1000, 9999)  # Bob

    # Public keys
    A = pow(g, a, p)
    B = pow(g, b, p)

    # Shared secret
    shared_secret_A = pow(B, a, p)
    shared_secret_B = pow(A, b, p)

    assert shared_secret_A == shared_secret_B  # sanity check

    return shared_secret_A


# AES ENCRYPTION / DECRYPTION
def generate_aes_key(shared_secret):
    # Convert integer to bytes and hash it to get a 256-bit key
    secret_bytes = str(shared_secret).encode()
    key = hashlib.sha256(secret_bytes).digest()
    return key


def aes_encrypt(message, key):
    iv = get_random_bytes(16)  # 16 bytes IV for AES-CBC
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(message.encode(), AES.block_size))
    return iv + ciphertext  # prepend IV for decryption


def aes_decrypt(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ct), AES.block_size)
    return plaintext.decode()


# MAIN FUNCTION
if __name__ == "__main__":
    # Publicly known parameters (large prime and primitive root)
    p = 0xFFFFFFFB  # large 32-bit prime
    g = 5

    # Step 1: Performing Diffie–Hellman exchange
    shared_secret = diffie_hellman_key_exchange(p, g)
    print("Shared Secret (integer):", shared_secret)

    # Step 2: Deriving AES key
    aes_key = generate_aes_key(shared_secret)
    print("Derived AES Key (SHA-256):", aes_key.hex())

    # Step 3: Encrypting and decrypting
    message = "Hello, World!"
    print("\nOriginal Message:", message)

    encrypted = aes_encrypt(message, aes_key)
    print("Encrypted Message (hex):", encrypted.hex())

    decrypted = aes_decrypt(encrypted, aes_key)
    print("Decrypted Message:", decrypted)


Shared Secret (integer): 2852378311
Derived AES Key (SHA-256): f172da2b5b0d5bee6237d577a6d5a045a1a35f0ee48a6f8265cddbc7260f50ae

Original Message: Hello, World!
Encrypted Message (hex): be8e5b1a0769c4470e34182be490b0e821cad7cd76d8b10752cd83dd4480f423
Decrypted Message: Hello, World!
